<a href="https://colab.research.google.com/github/isaacpy-cloud/TraderWolfs/blob/main/indicator_settings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Colab Destekli, İndikatör ve Hesaplama Botu**

In [3]:
!pip install python-binance
!pip install catboost
!pip install matplotlib --upgrade
!pip install mpl_finance

     |████████████████████████████████| 65.7MB 64kB/s 
     |████████████████████████████████| 11.6MB 4.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [16]:
import pandas as pd
import numpy as np
import time
import dateparser
import pytz
import json
import datetime as dt
from datetime import datetime, timedelta
from threading import Thread
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ochl
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from catboost import CatBoost, CatBoostClassifier, CatBoostRegressor, Pool, cv
from tqdm import tqdm as tqdm
from binance.client import Client

In [17]:
def moving_avarage(df, n):
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))
  df = df.join(MA)
  return df

In [18]:
def exponantial_moving_average(df,n):
  EMA = pd.Series(df['Close'],ewm(span=n, min_periods=n).mean(), name='EMA_' + str(n))
  df = df.join(EMA)
  return df

pd.to_datetime = Evrensel saniyelikten normal zamana çevirme

In [35]:
def binance_data(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','Taker Buy Base Asset Volume','Taker buy quote asset volume','Can be ignored'])
  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
  return df

In [36]:
def date_to_millisecond(date_Str):
  epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
  d = dateparser.parse(date_Str)
  if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
    d = d.replace(tzinfo=pytz.utc)
  return int((d- epoch).total_seconds()*1000.0)

In [37]:
def interval_to_milliseconds(interval):
  ms = None
  seconds_per_unit = {
      "m": 60,
      "h": 60 * 60,
      "d": 24 * 60 * 60,
      "w": 7 * 24 * 60 * 60
  }
  unit = interval[-1]
  if unit in seconds_per_unit:
    try:
      ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
    except:
      pass
  return ms

In [38]:
def binance_historical_klines(symbol, interval, start_str, end_str=None):
  output_data = []
  limit = 500
  timeFrame = interval_to_milliseconds(interval)
  start_ts = date_to_millisecond(start_str)
  end_ts = None
  if end_str:
    end_ts = date_to_millisecond(end_str)
  idx = 0
  symbol_existed = True
  while True:
    temp_data = client.get_klines(symbol=symbol,interval=interval,limit=limit,startTime=start_ts,endTime = end_ts)
    if not symbol_existed and len(temp_data):
      symbol_existed = True
    
    if symbol_existed:
      output_data += temp_data
      start_ts = temp_data[len(temp_data) - 1][0] + timeFrame
    
    idx += 1
    if len(temp_data) < limit:
      break
    if idx % 3 == 0:
      time.sleep(1)
  return output_data

## **Binans Api Bağlantıları**

In [39]:
api_key = 'hXjL8ujRBXbXLNaek31Z4ysCiCmfhnbRiVdApFFt8YlwBST6hUx1gpoVC4r5xjpS'
api_secret = 'zFyTQoSDZVdRvmmFnHgjP4oaPWcXC4hhBbICaavRKY4pBDKJawWOPX7hCF9uSAMw'

client = Client(api_key, api_secret)

In [40]:
symbol = "BTCTRY"
start = "1 Jan, 2017"
interval = Client.KLINE_INTERVAL_1DAY
klines = binance_historical_klines(symbol,interval,start)
klines = np.array(klines)
df = binance_data(klines)

In [41]:
df

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker buy quote asset volume,Can be ignored
0,2019-12-20,42330.0,42800.0,42066.0,42210.0,1.092721,2019-12-20 23:59:59.999,4.629721e+04,70.0,0.346199,1.469715e+04,0.0
1,2019-12-21,42490.0,42547.0,42203.0,42539.0,0.975067,2019-12-21 23:59:59.999,4.131325e+04,28.0,0.236337,1.003039e+04,0.0
2,2019-12-22,42415.0,44650.0,42400.0,44416.0,1.264342,2019-12-22 23:59:59.999,5.574385e+04,56.0,0.421050,1.842735e+04,0.0
3,2019-12-23,44579.0,45349.0,42581.0,43500.0,2.524750,2019-12-23 23:59:59.999,1.117467e+05,150.0,0.652482,2.926497e+04,0.0
4,2019-12-24,43596.0,44000.0,42103.0,43126.0,2.191588,2019-12-24 23:59:59.999,9.568290e+04,50.0,0.797389,3.456564e+04,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
435,2021-02-27,346403.0,359429.0,338500.0,346010.0,392.438623,2021-02-27 23:59:59.999,1.378716e+08,77342.0,189.060194,6.640985e+07,0.0
436,2021-02-28,346119.0,350100.0,328000.0,338979.0,384.642484,2021-02-28 23:59:59.999,1.305639e+08,46027.0,206.965053,7.029638e+07,0.0
437,2021-03-01,339000.0,362340.0,338373.0,360446.0,313.973966,2021-03-01 23:59:59.999,1.107305e+08,31968.0,148.789219,5.247698e+07,0.0
438,2021-03-02,360799.0,362497.0,349407.0,357704.0,298.782540,2021-03-02 23:59:59.999,1.064373e+08,28614.0,147.608663,5.259769e+07,0.0


In [42]:
remove_cols = [c for c in df.columns if c not in ['Open','Close','High','Low','Volume']]
df.drop(remove_cols, axis=1, inplace=True)
df

,Open,High,Low,Close,Volume
0,42330.0,42800.0,42066.0,42210.0,1.092721
1,42490.0,42547.0,42203.0,42539.0,0.975067
2,42415.0,44650.0,42400.0,44416.0,1.264342
3,44579.0,45349.0,42581.0,43500.0,2.524750
4,43596.0,44000.0,42103.0,43126.0,2.191588
...,...,...,...,...,...
435,346403.0,359429.0,338500.0,346010.0,392.438623
436,346119.0,350100.0,328000.0,338979.0,384.642484
437,339000.0,362340.0,338373.0,360446.0,313.973966
438,360799.0,362497.0,349407.0,357704.0,298.782540
